## Initialisation

In [1]:
# Path
SPECTRADIR <- 'C:/DATA/500_AD/S13-AA5'
QSDIR <- 'C:/DATA/500_AD/QC-QS'
METADATADIR <- 'C:/DATA/500_AD'
PROFILEDIR <- METADATADIR
OUTDIR <- 'tmp'
# Sub-directories of the QC/QS
QSname <- 'GLBBI_RB_AJdos-S13_QR20'

# Types of Instrument/Sequence/Wine
Field <- '500'
Type <- 'Red'
Sequence <- 'noesy';

# Filenames
PROFILE_FILENAME <- paste0(paste(c('profile-AJdos',Type,Sequence,Field),collapse='-'),'.txt')
STDS_FILENAME <- 'standards-QC-QS.txt'
SAMPLE_FILENAME <- 'samples-S13.txt' # see JNB sample-table 

# Results filename
XLSOUTPUT_FILENAME <- paste0('500_AD-S13-',Type,'_',Sequence,'_Results.xlsx')

# targeted compounds / zones
# --------------------------------------------------------------------
#cmpdlist <- c('acetoin', 'choline', 'fumaric acid','maleic acid')
#cmpdlist <- c('valine', 'isoleucine', 'threonine','citric acid', 'myo-inositol')
cmpdlist <- 'fructose'
zones <- NULL

## Processing

In [2]:
# Load packages : create then popuplate a new RnmrQuant1D instance 
library(RnmrQuant1D)
rq1d <- RnmrQuant1D$new()
rq1d$OUTTYPE <- "html"

# Display the spectra path / infos
cat("Raw Spectra Path:", SPECTRADIR,"\n")
cat("QS Spectra Path:", paste0(QSDIR,'/',QSname),"\n")
cat('Instrument Field =',Field,', Wine Type =',Type,', Sequence =',Sequence,"\n")

# Read the profile file - name in the form of profile-<Type>-<Sequence>-<Field>.txt
PROFILE <- file.path(PROFILEDIR, PROFILE_FILENAME)
cat("Profile:", PROFILE,"\n")
profile <- rq1d$readProfile(PROFILE)

# Read the calibration profile
STDS_FILE <- file.path(METADATADIR,STDS_FILENAME)
cat("Calibration Profile:", STDS_FILE,"\n")
stds_profile <- data.frame(read.table(STDS_FILE, header=T, sep="\t", dec=".", stringsAsFactors=F))

# Read the sample file
SAMPLEFILE  <- file.path(METADATADIR, SAMPLE_FILENAME)
cat("Samples:", SAMPLEFILE,"\n")
samples <- utils::read.table(SAMPLEFILE, sep="\t", header=T,stringsAsFactors=FALSE)
cat('Nb samples =',nrow(samples),"\n\n")

# Output Directories
LOGDIR <- file.path(OUTDIR,'log');        if (! dir.exists(LOGDIR) )   dir.create(LOGDIR, showWarnings = FALSE)
RDATADIR <- file.path(OUTDIR,'RData');    if (! dir.exists(RDATADIR) ) dir.create(RDATADIR, showWarnings = FALSE)
RDATADIR <- file.path(RDATADIR,Sequence); if (! dir.exists(RDATADIR) ) dir.create(RDATADIR, showWarnings = FALSE)

# Full path of the results file
XLSOUTPUT <- file.path(METADATADIR,XLSOUTPUT_FILENAME)

#-----------------

# Populate rq1d object
rq1d$RAWDIR <- SPECTRADIR
rq1d$QSDIR <- QSDIR
rq1d$TMPDIR <- LOGDIR
rq1d$RDATADIR <- RDATADIR
rq1d$PROFILE <- profile
rq1d$CALIBRATION <- stds_profile
rq1d$SAMPLES <- samples
rq1d$SEQUENCE <- Sequence
rq1d$FIELD <- Field
rq1d$TYPE <- Type

# Check if data is OK
cat("----\n")
rq1d$check_samples(verbose=TRUE)
rq1d$check_calibration(QS=QSname, verbose=TRUE)
rq1d$check_profile(verbose=TRUE)
rq1d$check_outdir(verbose=TRUE)
cat("----\n\n")

# Compute the ERETIC factor
# --------------------------------------------------------------------
rq1d$proc_fPULCON(QSname, thresfP=6, deconv=TRUE, verbose=1)

# Calculate for all samples the concentration of each of the targeted compounds based on their sequence
# --------------------------------------------------------------------
rq1d$proc_Quantification(cmpdlist, zones, ncpu=6, reset=TRUE, CR=TRUE, verbose=1)

# Save result tables
filelist <- list(SAMPLEFILE=SAMPLEFILE, PROFILE=PROFILE, CALIBRATION=STDS_FILE)
rq1d$save_Results(XLSOUTPUT, filelist)
cat("Results saved into",XLSOUTPUT,"\n")

# Save rq1d objet
save(rq1d, file=file.path(RDATADIR,"RQ1D.RData"))
cat("Save the RnmrQuant1D instance (rq1d object) into",file.path(RDATADIR,"RQ1D.RData"),"\n")

Raw Spectra Path: C:/DATA/500_AD/S13-AA5 
QS Spectra Path: C:/DATA/500_AD/QC-QS/GLBBI_RB_AJdos-S13_QR20 
Instrument Field = 500 , Wine Type = Red , Sequence = noesy 
Profile: C:/DATA/500_AD/profile-AJdos-Red-noesy-500.txt 
Calibration Profile: C:/DATA/500_AD/standards-QC-QS.txt 
Samples: C:/DATA/500_AD/samples-S13.txt 
Nb samples = 15 

----
OK: C:/DATA/500_AD/S13-AA5 contains some Bruker spectra acquired with a noesy sequence
OK: all spectra reported in the sample table appear to match a spectrum under C:/DATA/500_AD/S13-AA5
OK: the sample table has at least 4 columns with one named 'F_dilution'
OK: C:/DATA/500_AD/QC-QS contains some Bruker spectra acquired with a noesy sequence
OK: the standard profile format seems correct
OK: some spectra under C:/DATA/500_AD/QC-QS correspond to GLBBI_RB_AJdos-S13_QR20
OK: the quantification profile format seems correct
OK: output directories are correctly defined
----

Compute the PULCON factor ...
fP = 117122 ,  CV = 0.65 
elapsed time = 1.57 

Do